# 04 — spaCy (Deutsch): Tokens, Lemmata, POS, Adjektive

Ziel: Den Mixing‑Korpus linguistisch untersuchen. Wir extrahieren Token, Lemmata, Wortarten (POS), Adjektive für Sound‑Beschreibungen und einfache Phrasen.

**Was du lernst**
- Robust *de_core_news_sm* laden (venv/HF‑kompatibel)
- Tokenisierung & Lemmatisierung
- POS‑Tags (insb. Adjektive) und Noun‑Chunks
- Mini‑Auswertung (Top‑Adjektive, Häufigkeiten)


## Setup & Daten
Wir laden den Korpus aus `data/sample_corpus.json`. Wenn die Datei fehlt, verwenden wir einen kleinen Fallback. spaCy laden wir robust (direkter Paket‑Import **oder** `spacy.load`).

In [1]:
import json, importlib.util
from pathlib import Path
import pandas as pd
from collections import Counter

DATA = Path("data"); DATA.mkdir(exist_ok=True)

def load_corpus():
    p = DATA/"sample_corpus.json"
    if p.exists():
        with p.open("r", encoding="utf-8") as f:
            x = json.load(f)
            if isinstance(x, list) and all(isinstance(t, str) for t in x):
                return x
    return [
        "Die Snare ist zu laut und harsch",
        "Kick zu weich, es fehlt der Punch",
        "Vocals klingen nasal, 800 Hz absenken",
        "Bass maskiert die Kick, Sidechain nötig",
        "S-Laute sind scharf, De-Esser einsetzen",
    ]

texts = load_corpus()
len(texts), texts[:2]

(5, ['Die Snare ist zu laut und harsch', 'Kick zu weich, es fehlt der Punch'])

## spaCy robust laden
Wir versuchen zuerst den direkten Modul‑Import `de_core_news_sm` (funktioniert gut in virtuellen Umgebungen und auf Hugging Face). Fällt das aus, nutzen wir `spacy.load('de_core_news_sm')`.

In [2]:
def load_spacy_de():
    try:
        import spacy
        if importlib.util.find_spec("de_core_news_sm") is not None:
            import de_core_news_sm
            return de_core_news_sm.load()
        return spacy.load("de_core_news_sm")
    except Exception as e:
        raise RuntimeError(
            "spaCy DE nicht verfügbar. Installiere lokal: \n"
            "  python -m spacy download de_core_news_sm\n\n"
            f"Originalfehler: {e}"
        )

nlp = load_spacy_de()
nlp

## Ein Dokument inspizieren
Wir betrachten ein Beispiel und zeigen Token mit **Lemma** und **POS**.

In [3]:
doc = nlp(texts[0])
rows = [{"text": t.text, "lemma": t.lemma_, "pos": t.pos_} for t in doc]
pd.DataFrame(rows)

,text,lemma,pos
0,Die,der,DET
1,Snare,Snare,NOUN
2,ist,sein,AUX
3,zu,zu,PART
4,laut,laut,ADV
5,und,und,CCONJ
6,harsch,harsch,ADV


## Adjektive extrahieren (Sound‑Wörter)
Wir holen alle Adjektive pro Dokument und bauen eine globale Häufigkeitsliste.

In [4]:
def adjectives_per_doc(texts):
    out = []
    for tx in texts:
        doc = nlp(tx)
        adjs = [t.lemma_.lower() for t in doc if t.pos_ == "ADJ" and not t.is_stop]
        out.append(adjs)
    return out

adjs_list = adjectives_per_doc(texts)
Counter([a for lst in adjs_list for a in lst]).most_common()

[('nasal', 1)]

## Noun‑Chunks (einfache Phrasen)
Praktisch, um Phrasen wie *"mehr Raumanteil"* zu erkennen.

In [5]:
def noun_chunks(text):
    doc = nlp(text)
    return [nc.text for nc in doc.noun_chunks]

[(t, noun_chunks(t)) for t in texts]

[('Die Snare ist zu laut und harsch', ['Die Snare']),
 ('Kick zu weich, es fehlt der Punch', ['Kick', 'der Punch']),
 ('Vocals klingen nasal, 800 Hz absenken', ['800 Hz']),
 ('Bass maskiert die Kick, Sidechain nötig',
  ['Bass', 'die Kick', 'Sidechain']),
 ('S-Laute sind scharf, De-Esser einsetzen', ['S-Laute', 'De-Esser'])]

## Übersichtstabelle: Top‑Adjektive
Aggregierte Adjektive als DataFrame, sortiert nach Häufigkeit.

In [6]:
adj_counts = Counter([a for lst in adjs_list for a in lst])
adj_df = pd.DataFrame(sorted(adj_counts.items(), key=lambda x: -x[1]), columns=["adjective", "count"])\
         .reset_index(drop=True)
adj_df.head(20)

,adjective,count
0,nasal,1


## Übungen
1. Füge 5–10 weitere Textzeilen in `data/sample_corpus.json` hinzu und beobachte, wie sich die Top‑Adjektive ändern.
2. Filtere Adjektive nach *Sound‑Semantik* (z. B. *scharf, harsch, weich, punchy*) und zähle nur diese.
3. Extra: Füge eine Heuristik hinzu, die *Adjektiv + Substantiv* Phrasen extrahiert (z. B. *scharfe Hi‑Hats*).